In [ ]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("beauty_4/train_id_data.pickle","rb") as fr:
    train_data = pickle.load(fr)

with open("beauty_4/valid_id_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open("beauty_4/test_id_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [ ]:
with open("beauty_4/train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open("beauty_4/valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open("beauty_4/test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [ ]:
# ID
train_data[0] = tf.convert_to_tensor(train_data[0], dtype=tf.float32)
train_data[1] = tf.convert_to_tensor(train_data[1], dtype=tf.float32)

## Model

In [ ]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add, Multiply
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 4

In [ ]:
# set parameters:
V = 249274 
embedding_dim = 50
max_len = 1 
filters_num = 64
kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

In [ ]:
input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d_1 = Conv1D(filters=filters_num,
                kernel_size=kernel_size_1,
                activation='relu',
                padding='same')
conv1d_2 = Conv1D(filters=filters_num,
                kernel_size=kernel_size_2,
                activation='relu',
                padding='same')
conv1d_3 = Conv1D(filters=filters_num,
                kernel_size=kernel_size_3,
                activation='relu',
                padding='same')
maxpool1d = MaxPool1D(max_len)
dense = Dense(15)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector_1 = TimeDistributed(conv1d_1)(input_vector)
validation_conv_1 = conv1d_1(validation_vector)

convolutional_vector_2 = TimeDistributed(conv1d_2)(input_vector)
validation_conv_2 = conv1d_2(validation_vector)

convolutional_vector_3 = TimeDistributed(conv1d_3)(input_vector)
validation_conv_3 = conv1d_3(validation_vector)

convolutional_merge = Multiply()([convolutional_vector_1, convolutional_vector_2, convolutional_vector_3])
validation_merge = Multiply()([validation_conv_1, validation_conv_2, validation_conv_3])

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_merge)
validation_maxpooling = maxpool1d(validation_merge)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [ ]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [ ]:
middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [ ]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mae', metrics=['mse', 'mae', 'mape'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 1)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 4, 1, 50)    12463700    ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        12463700    ['input_2[0][0]']            

In [ ]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 16s 15ms/step - loss: 2.1329 - mse: 7.8833 - mae: 2.1329 - mape: 63.3201 - val_loss: 0.8068 - val_mse: 2.1618 - val_mae: 0.8068 - val_mape: 46.9493
Epoch 2/30
256/256 [==============================] - 3s 14ms/step - loss: 0.7788 - mse: 1.9206 - mae: 0.7788 - mape: 43.8032 - val_loss: 0.8438 - val_mse: 1.9409 - val_mae: 0.8438 - val_mape: 45.4432
Epoch 3/30
256/256 [==============================] - 4s 14ms/step - loss: 0.7091 - mse: 1.5536 - mae: 0.7091 - mape: 38.2827 - val_loss: 0.8792 - val_mse: 1.8129 - val_mae: 0.8792 - val_mape: 44.1696
Epoch 4/30
256/256 [==============================] - 4s 14ms/step - loss: 0.6573 - mse: 1.3523 - mae: 0.6573 - mape: 35.0002 - val_loss: 0.9039 - val_mse: 1.7962 - val_mae: 0.9039 - val_mape: 43.7793
Epoch 5/30
256/256 [==============================] - 4s 14ms/step - loss: 0.6302 - mse: 1.2661 - mae: 0.6302 - mape: 33.2417 - val_loss: 0.9239 - val_mse: 1.7968 - val_mae: 0.9239 - val_mape: 43

In [ ]:
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 8s 3ms/step - loss: 0.9745 - mse: 1.9767 - mae: 0.9745 - mape: 44.4650
[0.9745349287986755, 1.9767054319381714, 0.9745349287986755, 44.464969635009766]
